In [59]:
from dotenv import load_dotenv
load_dotenv()
from IPython.display import display,Markdown
def md(text:str):
    return display(Markdown(text))

## 1. `Simple` Implementation using `Class Model`

In [60]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

In [61]:
# Step-1 : Model Setup
model = ChatGroq(model="qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001F602D63F00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F602E4C7C0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

#### SystemMessage is okay here because the text is static (doesn't change)
    SystemMessage(content="You are a Helpful assistant"),
    
    MessagesPlaceholder(variable_name="history"),
    
    # You MUST use the PromptTemplate class to handle variables like {query}
    HumanMessagePromptTemplate.from_template("{query}")

#### ("role", "content") (recommaned)
    ("system", "You are a Helpful assistant"),
    
    # Placeholder for memory
    MessagesPlaceholder(variable_name="history"),
    
    # Use brackets {} to define a variable
    ("human", "{query}")

In [62]:
# 2. Create a prompt that expects history
from langchain.messages import SystemMessage,HumanMessage
prompt = ChatPromptTemplate.from_messages([
    # ("role", "content")
    ("system", "You are a Helpful assistant"),
    
    # Placeholder for memory
    MessagesPlaceholder(variable_name="history"),
    
    # Use brackets {} to define a variable
    ("human", "{query}")
])

In [63]:
# 3. Create the Chain (LCEL style)
chain = prompt | model

In [64]:
# 4. Setup Memory Management
# In a real app, 'store' would be a database (Redis, Postgres).
# For now, we use a simple dictionary.

store = {}
def get_session_history(session_id:str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [65]:
from rich import  print as pp
pp(store)

{}

In [66]:
# 5. Wrap the chain with Message History
chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="query",
    history_messages_key="history"
)
chain_with_memory

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x000001F602E0D260>, input_messages_key='query', history_messages_key='history', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

- Usage of Model class

In [67]:
res1 = chain_with_memory.invoke(
    {
        "query":"hi my name is ash!"
    },
    config={
        "configurable":{
            "session_id":"user_1"
        }
    }
)

In [68]:
md(res1.content)

<think>
Okay, the user just said, "hi my name is ash!" I need to respond appropriately. Let's see, the user introduced themselves, so I should acknowledge that and respond in a friendly manner. I should make sure to use their name in the response to create a personal connection.

First, I'll start with a greeting like "Hi Ash!" to confirm their name. Then, maybe add a cheerful emoji to keep the tone positive. Next, I should offer assistance, so something like "How can I help you today?" That invites them to ask questions or share what they need. I should also mention that I'm here to chat, so they know the conversation can be casual. 

I need to check for any typos and ensure the response is concise. Maybe add another emoji at the end to keep it friendly, like a smiley or sparkles. Let me put it all together: "Hi Ash! 😊 How can I help you today? Feel free to ask me anything or just chat!" That should cover it. Double-checking the name is correct and the tone is welcoming.
</think>

Hi Ash! 😊 How can I help you today? Feel free to ask me anything or just chat! ✨

In [69]:
res2 = chain_with_memory.invoke(
    {
        "query":"What is my name?"
    },
    config={
        "configurable":{
            "session_id":"user_1"
        }
    }
)

In [70]:
md(res2.content)

<think>
Okay, the user asked, "What is my name?" So first, I need to recall the previous conversation. The user introduced themselves as Ash. My last response was confirming their name and offering help. Now they're asking again.

Hmm, maybe they want to test if I remember their name. Let me check the history. Yes, they said "hi my name is ash!" in the first message. I acknowledged it. So I should respond with their name again, but maybe add something to keep the conversation going. They might be checking if the AI retains context. 

I should make sure to mention their name clearly and ask how I can assist them further. Also, maybe add a friendly emoji to keep the tone positive. Let me phrase it as a question to encourage them to respond. Avoid any confusion. Keep it simple and friendly.
</think>

Hi again, **Ash**! 😊 How can I assist you today? I'm here to help with anything you need! ✨

In [71]:
res3 = chain_with_memory.invoke(
    {
        "query":"I am student of government college of engeniering , i have 2 girl friends and 6 female friends . i am poor so i have sugar mommy to take care of my expenses , she is very nice to me never scolds me for anything and also teaches me maths and german language."
    },
    config={
        "configurable":{
            "session_id":"user_1"
        }
    }
)
md(res3.content)

<think>
Okay, the user has shared some personal information. Let me break it down. He's a student at Government College of Engineering, has 2 girl friends and 6 female friends. He's poor and has a sugar mommy who helps with expenses, is nice, teaches him math and German. 

First, I need to acknowledge his situation. He might feel lonely or maybe he's relying on the sugar mommy for support. He could be looking for validation or advice. It's important to be supportive without judgment.

He mentioned friends, but maybe he's trying to build connections. The sugar mommy relationship is a bit complex. I should consider that there might be underlying issues, like dependency or potential exploitation. But I shouldn't assume; instead, offer a safe space for him to talk.

He might be feeling grateful towards the sugar mommy but also conflicted. It's good to encourage him to explore other support systems, like formal education resources or campus services. Maybe he's also looking for ways to become more self-sufficient.

I should respond with empathy, affirm his gratitude, and subtly suggest exploring other avenues for support. Make sure to ask if he needs help with anything specific, so he feels heard and supported without pressure.
</think>

It’s great to hear more about your life, Ash! 🌟 Your situation is unique, and it’s clear you’re working hard to balance school, friendships, and personal growth. Here are a few thoughts and questions to consider:

1. **Gratitude and Relationships**: It sounds like your sugar mommy is providing significant emotional and financial support, which can be both a blessing and a complex dynamic. Are you comfortable with this arrangement, or do you ever feel conflicted about it?

2. **Friendships**: You mentioned having 2 girlfriends and 6 female friends. How do these relationships support you emotionally or socially? Are there challenges in maintaining these connections while managing your studies and expenses?

3. **Education and Goals**: You’re studying engineering, which is demanding. How do you feel about your progress in your studies? The German language and math skills your sugar mommy teaches you—do you see these skills helping you in your future career or personal growth?

4. **Financial Independence**: You mentioned being "poor." Are there resources at your college (scholarships, part-time work, etc.) or community programs that could help you reduce reliance on financial support? It might be worth exploring opportunities to build long-term independence.

5. **Emotional Support**: It’s great that your sugar mommy is kind and patient, but do you have other mentors or confidants you can lean on for advice or encouragement? Having diverse sources of support can be beneficial.

If you’re comfortable sharing more, I’d be happy to help brainstorm solutions or discuss your goals further. Always remember to prioritize your well-being and set boundaries that feel right for you. 💪✨

In [72]:
res4 = chain_with_memory.invoke(
    {
        "query":"how many girlfriends and total female friends  do i have ? respond in numbers only like this gf -> X , total frnd -> Y"
    },
    config={
        "configurable":{
            "session_id":"user_1"
        }
    }
)
md(res4.content)

<think>
Okay, let me see. The user provided that information in their previous message.

They mentioned they have 2 girl friends and 6 female friends. Wait, the question is about girlfriends and total female friends. The user used "girlfriends" and "girl friends" which might be a bit confusing. In the initial message, they wrote "i have 2 girl friends and 6 female friends." So, "girl friends" likely refers to girlfriends, and "female friends" just female friends. 

So, the user wants the number of girlfriends (gf) and total female friends (total frnd). The 2 girl friends are the girlfriends, so gf -> 2. The total female friends would be the 2 girlfriends plus the 6 female friends, making it 8 total. But I need to confirm if "girl friends" and "female friends" are separate categories. The user might consider "girlfriends" as romantic partners and "female friends" as just friends. Therefore, the total female friends would be 2 + 6 = 8. 

I should respond with gf -> 2 and total frnd -> 8. But wait, maybe "girl friends" is a typo and they meant "girlfriends". Let me check again. The original message says: "i have 2 girl friends and 6 female friends". So "girl friends" are 2, which are likely girlfriends, and "female friends" are 6. So total female friends would be 2 + 6 = 8. 

Yes, that makes sense. The user wants the count of girlfriends (gf) as 2 and total female friends (total frnd) as 8. I need to make sure the response is in the format they specified: numbers only, like gf -> X, total frnd -> Y. 

So the answer should be: gf -> 2, total frnd -> 8.
</think>

gf -> 2, total frnd -> 8

## 2. `Simple` Implementation using `agent`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.agents 